In [1]:
import multiprocessing
import gensim
from gensim.models.word2vec import Word2Vec
import nltk
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

c:\programdata\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Taller 03: Word2Vec

## Alumnos: 
- Alvarez Muñoz Carolina
- Escobar Aldo 

## Objetivo

El objetivo de este taller es doble. En primer lugar que estudien por su cuenta qué es lo que hacen los diferentes parámetros de word2vec en gensim, y en segundo lugar que usen word2vec y experimenten un poco con sus resultados.

## Parte 1

Lean la sección de word2vec del capítulo 6 de la última edición del libro de Jurafsky y Martin (https://web.stanford.edu/~jurafsky/slp3/6.pdf) y también busquén posts online referidos a word2vec (por ej. http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/, http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/, https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial). Una vez que hayan hecho esto respondan en no más de tres oraciones qué rol cumple cada uno de estos parámetros.

* **min_count**: Es un integer que funciona como un valor "techo" con el fin de que el modelo ignore todas las palabras con una frecuencia inferior a este valor. Las palabras extremadamente infrecuentes generalmente no son importantes, por lo que es mejor deshacerse de ellas, a menos que el data set sea realmente pequeño, esto no afectaría al modelo.
* **window**: Es un integer que determina cuántas palabras antes y después de una palabra dada se incluirían como palabras de contexto de la palabra en cuestión. Valores de ventana chicos dan resultados de naturaleza más sintáctica, en cambio valores más grandes dan resultados de naturaleza más semántica. Este último lleva más tiempo de training dado que el tamaño de la ventana es más grande.
* **size**: Es un integer que representa la dimensionalidad del vector. Cuanto mayor sea el número de dimensiones que está utilizando, más información se puede almacenar. Los modelos vectoriales densos tienen una dimensionalidad de 50-300 y las dimensiones son más difíciles de interpretar.
* **sample**: Es un float que representa el umbral que permite configurar las palabras con mayor frecuencia en los datos de entrenamiento para que sean muestreados aleatoriamente. El valor predeterminado es 1e-3, el rango útil es (0, 1e-5) (Ver: https://github.com/RaRe-Technologies/gensim/blob/727caf220ba2ebe253beaee73877afbd99b2db02/gensim/models/word2vec.py#L600)
* **negative**: Es un integer que representa el número de ejemplos negativos (muestras de palabras que no son vecinas), por defecto su valor es 5 y los valores comunes son entre 3 a 10. Si su valor es 0, no se utiliza muestreo negativo. Esta idea de muestreo negativo está inspirada en este [paper](http://proceedings.mlr.press/v9/gutmann10a/gutmann10a.pdf) sobre estimación de contraste de ruido, por lo que estamos constrastando la señal real (ejemplos positivos de palabras vecinas) con ruido (palabras seleccionadas al azar que no son vecinas), lo que permite [una gran compensación de eficiencia computacional y estadística](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling).  La idea del muestreo negativo es que si el modelo puede distinguir entre los pares probables (positivos) frente a los pares improbables (negativos), se aprenderán buenos vectores de palabras.
* **ns_exponent**: Es un float que representa el exponente utilizado para dar forma a la distribución de muestreo negativa. Un valor de 1.0 muestrea exactamente en proporción a las frecuencias, 0.0 muestrea todas las palabras por igual, mientras que un valor negativo muestrea más palabras de baja frecuencia que palabras de alta frecuencia. El popular valor predeterminado de 0.75 fue elegido por el paper original de Word2Vec. Algunas investigaciones sugieren que otras aplicaciones, como los sistemas de recomendación, podrían beneficiarse de otros valores.

## Parte 2

En esta parte pueden seguir dos caminos:

1. Bajar 50000 notas de La Nación elegidas al azar (puede reutilizar las ~11000 entregadas en clase).
2. Utilizar un corpus de su elección (puede ser alguno que se encuentre disponible online o alguno que ustedes hayan creado -por ejemplo texto en foros de Rava Online, Artículos de Wikipedea-).

Utilizando el corpus elegido, entrenen dos modelos de word2vec, uno con una ventana pequeña y otro con una ventana grande (qué es grande y qué es pequeña queda a su criterio). Una vez entrenado evalue si los vectores encontrados tienen sentido (si los mismos captan bien similitud entre palabras y si logran captar analogías).

Para resolver este punto no deben entregar el código que **descarga** el corpus, si no que el código debe comenzar suponiendo que ya tienen el corpus a analizar en sus máquinas.

# RESOLUCIÓN

## Lectura y preparación del corpus de Quora Questions.
Preprocesamos el texto y filtramos los datos, quedandonos con aquellos que no están duplicados.

In [2]:
def read_questions(row,column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]).encode('utf-8'))

In [3]:
df = pd.read_csv("./dataQuora.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
len(df)

404290

In [5]:
documents = []
for index, row in df.iterrows():
    documents.append(read_questions(row,"question1"))
    if row["is_duplicate"] == 0:
        documents.append(read_questions(row,"question2"))

In [39]:
len(documents)

659317

In [6]:
documents[5]

['how',
 'can',
 'internet',
 'speed',
 'be',
 'increased',
 'by',
 'hacking',
 'through',
 'dns']

## Modelo 1 (WINDOW = 1)

En este modelo intentaremos probar window=1. Efectivamente, como mencionamos en su definición, se ve claramente que el modelo logra "aprender" sintácticamente el corpus sobre el cual fue entrenado. 

En ambos modelos se entreno con sg=1 (Skip-Gram) ya que brindó mejores resultados que con sg=0.


In [7]:
# Defino los parámetros del modelo
model1 = gensim.models.Word2Vec(documents, size=100,
                               window=1,
                               min_count=1,
                               workers=multiprocessing.cpu_count(), sg=1)

In [8]:
print('Model vocab size : ', len(model1.wv.vocab))

Model vocab size :  78611


### Similitud entre 2 palabras
Chequeamos que el primer modelo detecta alta similitud entre django y flask, entre scala y python y entre csv y json, al igual que el modelo 2. Pero para los pares (phonegap, mobile) el modelo no detecta mucha similitud, con respecto al modelo 2. Pasa al revés con el par (holden,spark)  

In [9]:
model1.wv.similarity('django', 'flask')

0.86805457

In [10]:
model1.wv.similarity('scala', 'python')

0.8533266

In [11]:
model1.wv.similarity("csv", "json")

0.9063615

In [12]:
model1.wv.similarity("phonegap", 'mobile')

0.5908629

In [13]:
model1.wv.similarity("holden", "spark")

0.4928444

### Palabras mas similares
Con respecto al modelo 2, difiere mucho en udemy, testing y phonegap.
- El modelo 2, para phonegap, identificó muy bien a xamarin como tecnología alternativa. Tambien sdk (termino muy relacionado  a mobile) y prototyping (uno de sus principales usos)
- El modelo 2 para udemy capto mas plataformas educativas online que el modelo 1.
- El modelo 2 para aws captura a Azure, en cambio el modelo 1 no.
- El modelo 1 para testing capta mejor ciertos procesos, etapas y actividades de la Ingenieria del Software, en cambio el modelo 2 capta mejor ciertas tecnologías o terminologías relacionadas a la palabra 'testing'
- Tanto el modelo 1 como el modelo 2 difieren en cuanto al listado de terminos similares a Argentina. En el modelo 2 los 2 primeros términos corresponden a paises que también son latinoamericanos.

Con el resto de los términos, ambos modelos capturan más o menos lo mismo.

In [14]:
model1.wv.most_similar(positive=['json', 'python'], negative=['java'])

[('csv', 0.856725811958313),
 ('xml', 0.8416116237640381),
 ('rar', 0.8413346409797668),
 ('psd', 0.8217413425445557),
 ('png', 0.8141417503356934),
 ('jpeg', 0.8069920539855957),
 ('mov', 0.8029743432998657),
 ('sketch', 0.7998624444007874),
 ('jpg', 0.7948867082595825),
 ('ost', 0.7894456386566162)]

In [15]:
model1.wv.most_similar(positive=['vpn', 'python'], negative=['java'])

[('slack', 0.7278505563735962),
 ('proxy', 0.7107540965080261),
 ('microphone', 0.7066342234611511),
 ('modem', 0.7057940363883972),
 ('ftp', 0.7034748792648315),
 ('applock', 0.6997796297073364),
 ('ssh', 0.6992812752723694),
 ('facetime', 0.6965632438659668),
 ('robinhood', 0.695682942867279),
 ('multiplayer', 0.6951815485954285)]

In [63]:
model1.most_similar('aws')

[('salesforce', 0.8383669853210449),
 ('etl', 0.819669246673584),
 ('cad', 0.8155735731124878),
 ('ec', 0.8135716319084167),
 ('heroku', 0.8088165521621704),
 ('evernote', 0.8062582612037659),
 ('crm', 0.8029599189758301),
 ('firebase', 0.7968751788139343),
 ('documentation', 0.792850136756897),
 ('tibco', 0.7865740060806274)]

In [62]:
model1.most_similar('facebook')

[('fb', 0.8690828084945679),
 ('twitter', 0.8590004444122314),
 ('linkedin', 0.8329737186431885),
 ('whatsapp', 0.8320820927619934),
 ('viber', 0.8299320936203003),
 ('tumblr', 0.8278658986091614),
 ('telegram', 0.8209035992622375),
 ('wechat', 0.8184382915496826),
 ('pinterest', 0.8146739602088928),
 ('instagram', 0.8110525608062744)]

In [17]:
model1.wv.most_similar('udemy')

[('coursera', 0.8572806119918823),
 ('ebooks', 0.8418262004852295),
 ('edx', 0.8337137699127197),
 ('nptel', 0.8174368143081665),
 ('lynda', 0.79087233543396),
 ('receipts', 0.7748346924781799),
 ('vimeo', 0.7724509239196777),
 ('matrimony', 0.7713868021965027),
 ('sharepoint', 0.7655866742134094),
 ('audiobook', 0.7635728120803833)]

In [18]:
model1.wv.most_similar('testing')

[('tester', 0.7317851781845093),
 ('prototyping', 0.714449405670166),
 ('development', 0.7115980982780457),
 ('hana', 0.7033167481422424),
 ('dba', 0.6967520713806152),
 ('deployment', 0.6956557631492615),
 ('erp', 0.6941887736320496),
 ('documentation', 0.6931678652763367),
 ('assurance', 0.6930036544799805),
 ('abap', 0.6901017427444458)]

In [19]:
model1.wv.most_similar('phonegap')

[('wix', 0.8774884939193726),
 ('netbeans', 0.8726273775100708),
 ('qwote', 0.8706647753715515),
 ('prestashop', 0.87028568983078),
 ('macros', 0.8671637177467346),
 ('meteorjs', 0.8665980696678162),
 ('codeblocks', 0.8653455972671509),
 ('squarespace', 0.864075243473053),
 ('integrating', 0.8634461164474487),
 ('fedora', 0.8633407354354858)]

In [44]:
model1.wv.most_similar('argentina')

[('belgium', 0.8738886117935181),
 ('cuba', 0.8719927072525024),
 ('bulgaria', 0.8643927574157715),
 ('colombia', 0.8631213903427124),
 ('uruguay', 0.8569871187210083),
 ('belize', 0.8569393754005432),
 ('chile', 0.8470370173454285),
 ('albania', 0.8469589352607727),
 ('sudan', 0.8456088304519653),
 ('austria', 0.844763994216919)]

### Palabra que no matchea en la lista
El modelo 1 detecta bien, en los 3 primeros casos, al igual que el modelo 2, aquellas palabras que no "matchean con el resto".
Para el caso de etl, aws y azure, lo hace distinto al modelo 2

In [20]:
model1.wv.doesnt_match(['python', 'sql', 'javascript', 'java'])

'sql'

In [21]:
model1.wv.doesnt_match(['python', 'vpn', 'javascript', 'java'])

'vpn'

In [22]:
model1.wv.doesnt_match(['beer', 'coffee', 'coding'])

'coding'

In [55]:
model1.wv.doesnt_match(['etl', 'aws', 'azure'])

'azure'

## SEGUNDO MODELO: (WINDOW = 4)

In [23]:
# Defino los parámetros del modelo
model2 = gensim.models.Word2Vec(documents, size=100,
                               window=4,
                               min_count=1,
                               workers=multiprocessing.cpu_count(), sg=1)

In [24]:
print('Model vocab size : ', len(model2.wv.vocab))

Model vocab size :  78611


### Similitud entre 2 palabras

In [25]:
model2.wv.similarity('django', 'flask')

0.86212575

In [26]:
model2.wv.similarity('scala', 'python')

0.867182

In [27]:
model2.wv.similarity("csv", "json")

0.8794539

In [28]:
model2.wv.similarity("phonegap", 'mobile')

0.6958955

In [29]:
model2.wv.similarity("holden", "spark")

0.23774163

### Palabras mas similares

In [30]:
model2.wv.most_similar(positive=['json', 'python'], negative=['java'])

[('xml', 0.8519490957260132),
 ('csv', 0.841950535774231),
 ('jpg', 0.8392014503479004),
 ('txt', 0.8230435848236084),
 ('hdfs', 0.8211490511894226),
 ('swf', 0.8166185617446899),
 ('psd', 0.8165850043296814),
 ('docx', 0.8154696822166443),
 ('png', 0.806637167930603),
 ('render', 0.8055713176727295)]

In [31]:
model2.wv.most_similar(positive=['vpn', 'python'], negative=['java'])

[('tor', 0.741836667060852),
 ('proxy', 0.7233006358146667),
 ('browser', 0.7063598036766052),
 ('blackberry', 0.6910176277160645),
 ('facetime', 0.688700795173645),
 ('ssh', 0.6809120178222656),
 ('spyware', 0.6794326305389404),
 ('comcast', 0.67850661277771),
 ('kodi', 0.6775163412094116),
 ('vizio', 0.6764388084411621)]

In [61]:
model2.most_similar('aws')

[('azure', 0.8567851781845093),
 ('firebase', 0.8309110403060913),
 ('qtp', 0.829558253288269),
 ('cloudera', 0.82160484790802),
 ('tableau', 0.8213474154472351),
 ('salesforce', 0.8168648481369019),
 ('wpf', 0.8149662613868713),
 ('etl', 0.8145946264266968),
 ('heroku', 0.8139879703521729),
 ('mongodb', 0.8123369216918945)]

In [60]:
model2.most_similar('facebook')

[('twitter', 0.8746172785758972),
 ('fb', 0.8691712617874146),
 ('linkedin', 0.8483709096908569),
 ('tumblr', 0.8357803821563721),
 ('whatsapp', 0.8323224782943726),
 ('pinterest', 0.8207857608795166),
 ('messenger', 0.8183181285858154),
 ('kik', 0.8156495094299316),
 ('telegram', 0.8020131587982178),
 ('instagram', 0.7952909469604492)]

In [33]:
model2.wv.most_similar('udemy')

[('lynda', 0.8603887557983398),
 ('ebooks', 0.8042945861816406),
 ('opencourseware', 0.802211344242096),
 ('edx', 0.7960515022277832),
 ('explainer', 0.787355899810791),
 ('marketers', 0.7821062803268433),
 ('coursera', 0.780627965927124),
 ('advertisers', 0.7779604196548462),
 ('vimeo', 0.7769426107406616),
 ('pornhub', 0.7749009132385254)]

In [34]:
model2.wv.most_similar('testing')

[('automation', 0.7882766723632812),
 ('selenium', 0.7638168334960938),
 ('etl', 0.7507907152175903),
 ('hadoop', 0.7422379851341248),
 ('methodologies', 0.729033887386322),
 ('qtp', 0.7264882326126099),
 ('dba', 0.7260955572128296),
 ('erp', 0.7231667637825012),
 ('pega', 0.7198524475097656),
 ('usability', 0.7187020182609558)]

In [35]:
model2.wv.most_similar('phonegap')

[('sandbox', 0.8888113498687744),
 ('ides', 0.8842485547065735),
 ('opensource', 0.8829988241195679),
 ('xamarin', 0.8826943635940552),
 ('prototyping', 0.8824082612991333),
 ('sdk', 0.8786871433258057),
 ('golang', 0.878308892250061),
 ('netbeans', 0.8751993179321289),
 ('sqlite', 0.8664655685424805),
 ('drupal', 0.8626308441162109)]

In [43]:
model2.wv.most_similar('argentina')

[('uruguay', 0.8431172370910645),
 ('colombia', 0.8374465107917786),
 ('italy', 0.8190811276435852),
 ('yemen', 0.8117990493774414),
 ('bulgaria', 0.8082664608955383),
 ('austria', 0.8056122064590454),
 ('denmark', 0.8030199408531189),
 ('chile', 0.801636815071106),
 ('japan', 0.7993309497833252),
 ('france', 0.7982182502746582)]

### Palabra que no matchea en la lista

In [36]:
model2.wv.doesnt_match(['python', 'sql', 'javascript', 'java'])

'sql'

In [37]:
model2.wv.doesnt_match(['python', 'vpn', 'javascript', 'java'])

'vpn'

In [38]:
model2.wv.doesnt_match(['beer', 'coffee', 'coding'])

'coding'

In [54]:
model2.wv.doesnt_match(['etl', 'aws', 'azure'])

'etl'

## Modalidad de entrega

El taller puede ser resuelto en grupos de a lo sumo 3 integrantes.

La resolución consiste en escribir una notebook que resuelva lo que la consigna pide.

Se evaluará que el código entregado sea claro y legible.

Si el código no corre, el taller no estará aprobado y deberá ser reentregado.

La entrega debe hacerse a rgalvez@utdt.edu.